## Agent에서 Function 도구 사용하기

#### 다중 턴 대화로 에이전트 실행

Azure OpenAI Chat Completion 서비스를 기반으로 구축된 에이전트에서 함수 도구를 사용하는 방법을 살펴봅니다

> **중요**
>
> 모든 에이전트 유형이 function calling을 지원하는 것은 아닙니다. 일부 에이전트는 사용자 지정 내장 도구만 지원하고 호출자가 자체 기능을 제공하는 것을 허용하지 않을 수 있습니다. 이 단계에서는 function calling을 지원하는 채팅 클라이언트를 통해 생성된 에이전트를 사용합니다.

#### Function Tool을 사용하는 에이전트 생성

Function 도구는 에이전트가 필요할 때 호출할 수 있도록 하려는 사용자 지정 코드입니다. 에이전트를 생성할 때 에이전트 tools 매개변수에 전달하면 모든 Python 함수를 Function 도구로 만들 수 있습니다 .

에이전트가 여러 함수 중에서 더 정확하게 선택할 수 있도록 함수 또는 해당 매개변수에 대한 추가 설명을 제공해야 하는 경우, Python의 타입 어노테이션 Annotated와 Pydantic의 Field를 사용하여 설명을 제공할 수 있습니다.

다음은 특정 위치의 날씨 정보를 가져오는 것처럼 보이는 간단한 함수 도구의 예입니다. 이 도구는 타입 어노테이션을 사용하여 함수와 위치 매개변수에 대한 추가 설명을 에이전트에 제공합니다.

In [1]:
from typing import Annotated
from pydantic import Field

def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

ai_function 데코레이터를 사용하여 함수의 이름과 설명을 명시적으로 지정할 수도 있습니다 .

In [1]:
from typing import Annotated
from pydantic import Field
from agent_framework import ai_function

@ai_function(name="weather_tool", description="Retrieves weather information for any location")
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    return f"The weather in {location} is cloudy with a high of 15°C."

> 참고 : ai_function 데코레이터에서 name 및 description 매개변수를 지정하지 않으면 프레임워크는 함수의 이름과 독스트링을 자동으로 대체값으로 사용합니다.

#### Function 도구를 사용하는 Agent 생성

In [2]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are a helpful assistant",
    tools=get_weather
)

이제 에이전트를 평소처럼 실행하기만 하면 에이전트가 필요할 때 get_weather 함수 도구를 호출할 수 있습니다 .

In [3]:
async def main():
    result = await agent.run("What is the weather like in Amsterdam?")
    print(result.text)

await main()

The weather in Amsterdam is currently cloudy with a high temperature of 15°C.


여러 기능 도구를 메서드로 포함하는 클래스를 만들 수도 있습니다. 이는 관련 기능들을 함께 정리하거나 기능들 간에 상태를 전달하려는 경우에 유용할 수 있습니다.

In [12]:
class WeatherTools:
    def __init__(self):
        self.last_location = None

    def get_weather(
        self,
        location: Annotated[str, Field(description="The location to get the weather for.")],
    ) -> str:
        """Get the weather for a given location."""
        self.last_location = location
        return f"The weather in {location} is cloudy with a high of 15°C."

    def get_weather_details(self) -> int:
        """Get the detailed weather for the last requested location."""
        if self.last_location is None:
            return "No location specified yet."
        return f"The detailed weather in {self.last_location} is cloudy with a high of 15°C, low of 7°C, and 60% humidity."

In [13]:
tools = WeatherTools()
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are a helpful assistant",
    tools=[tools.get_weather, tools.get_weather_details]
)

In [ ]:
# 새로운 쓰레드 생성
thread = agent.get_new_thread()

result = await agent.run("What is the weather like in Seoul?",thread=thread)
print("결과 1:" + result.text)

result2 = await agent.run("What is the detailed weather?",thread=thread)
print("결과 2:" + result2.text)

결과 1:The weather in Seoul is currently cloudy, with a high temperature of 15°C.
결과 2:Today in Seoul, the weather is cloudy. The temperature ranges from a high of 15°C to a low of 7°C, and the humidity is at 60%.
